In [ ]:
!pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
!pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [2]:
from snowflake.snowpark import Session

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4.gcp', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'test_database_prak', 'schema': 'test_schema_prak', 'role': 'PRAKHAR'}


In [7]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [1]:
from snowflake.ml.registry import model_registry

In [ ]:
!pip freeze | grep -i snow

In [8]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

True

In [9]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

TypeError: __init__() got an unexpected keyword argument 'database'

In [14]:
registry = model_registry.ModelRegistry(session=session, database_name="test_database_prak", schema_name="test_schema_prak")

SnowparkSQLException: (1304): 01b47ea0-0000-643f-0000-576d069c17da: 002043 (02000): SQL compilation error:
Object does not exist, or operation cannot be performed.

In [13]:
registry.get_history().show()

-------------------------------------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"  |"EVENT_ID"  |"MODEL_ID"  |"ROLE"  |"OPERATION"  |"ATTRIBUTE_NAME"  |"VALUE[ATTRIBUTE_NAME]"  |
-------------------------------------------------------------------------------------------------------------------
|                   |            |            |        |             |                  |                         |
-------------------------------------------------------------------------------------------------------------------

